In [1]:
import re
p = re.compile('[a-z]+')
d = p.match('dz')
print(d)

p2 = re.compile('\)\{')
d2 = p2.search('dz){')
print(d2)

<re.Match object; span=(0, 2), match='dz'>
<re.Match object; span=(2, 4), match='){'>


In [2]:
'''
    ['{' Count하는 Module 작성]
        If, 함수 선언의 경우 )로 구분이 가능하지만,
        else, struct의 경우는 )가 존재하지 않기에 문자로 구분해주어야 함

    이후에 추가해야 할 부분,,,
        1) "문자열" 부분 제외시키기
        2) 우선 이거로 진행하고, 이후에 (1) Function, Class, Struct (2) Condition (if, switch, else) (3) Iterative (for, while, do while,,,)
            구분지어서 Check해볼지도 생각해보기
'''

f = open('./test.c', 'r')
line_list = f.readlines()

p1 = re.compile('[\)|\S]\{') # 선언부에서 '{'를 열고, 공백이 존재하지 않는 경우
p2 = re.compile('[\)|\S]\s+\{') # 선언부에서 '{'를 열고, 공백이 반드시 존재하는 경우
p3 = re.compile('\s*\{') # 선언부에서 개행을 한 후 '{'를 여는 경우

#print('[line_list]')
#print(line_list)

p1_count = 0
p2_count = 0
p3_count = 0
for line in line_list:
    if p1.search(line) is not None:
        p1_count += 1
    elif p2.search(line) is not None:
        p2_count += 1
    elif p3.search(line) is not None:
        p3_count += 1

total_count = p1_count + p2_count + p3_count
print("Total:", total_count)
print("P1:", p1_count)
print("P2", p2_count)
print("P3", p3_count)

f.close()

Total: 7
P1: 3
P2 1
P3 3


In [3]:
'''
    [주석 Count하는 Module 작성]
        //{Comment} vs // {Comment} vs /* {Comment} */

    이후에 추가해야 할 부분,,,
        "문자열" 부분 제외시키기
        Nuke.c 에서 오검출이 나오는데, URL의 https:// 이게 주석이랑 헷갈리게 감지
'''

f = open('./test.c', 'r')
line_list = f.readlines()

p1 = re.compile('//\S') # //{Comment}: NO Space
p2 = re.compile('//\s+\S') # // {Comment}: Space
p3 = re.compile('/\*') # /* */ comment

#print('[line_list]')
#print(line_list)

p1_count = 0
p2_count = 0
p3_count = 0
for line in line_list:
    if p1.search(line) is not None:
        p1_count += 1
    elif p2.search(line) is not None:
        p2_count += 1
    elif p3.search(line) is not None:
        p3_count += 1

total_count = p1_count + p2_count + p3_count
print("Total:", total_count)
print("P1:", p1_count)
print("P2", p2_count)
print("P3", p3_count)

f.close()

Total: 9
P1: 5
P2 2
P3 2


In [8]:
'''
    [If, For, While, Switch + '(']
        Space vs No Space
    
    Condition과 Iterative를 나눌 필요는 없다고 생각함
    
    함수의 경우는 Space를 사용하는 경우가 극히 드물다고 생각.
        e.g. printf()처럼 쓰지 printf ()로 사용하진 않음

    이후에 추가해야 할 부분,,,
        "문자열" 부분 제외시키기
'''

f = open('./test.c', 'r')
line_list = f.readlines()

no_space = re.compile('if\(|switch\(|for\(|while\(') # if() or switch() or for() or while(): NO Space
with_space = re.compile('if\s+\(|switch\s+\(|for\s+\(|while\s+\(') # if () or switch () or for () or while (): With Space

no_count = 0
with_count = 0

for line in line_list:
    if no_space.search(line) is not None:
        no_count += 1
    elif with_space.search(line) is not None:
        with_count += 1

total_count = no_count + with_count
print("Total:", total_count)
print("NO Space:", no_count)
print("With Space", with_count)

f.close()

Total: 5
NO Space: 3
With Space 2


In [16]:
'''
    [Unary Operator]
        Unary: ++,--,+,-,~,!,(type)가 존재,
                위의 연산자 들 중 Space를 활용할만한 연산자는 (Type)뿐이라고 판단.
        
    이후에 추가해야 할 부분,,,
        "문자열" 부분 제외시키기
'''

f = open('./test.c', 'r')
line_list = f.readlines()

type_list = ['char', 'short', 'int', 'long', 'long long', 
            'unsigned char', 'unsigned short', 'unsigned int', 'unsigned long',
            'float', 'double', 'long double']

no_count = 0
with_count = 0

for line in line_list:
    for data_type in type_list:
        no_space = re.compile(rf'\({data_type}\)\S')
        with_space = re.compile(rf'\({data_type}\)\s+\S')
        
        if no_space.search(line) is not None:
            no_count += 1
        elif with_space.search(line) is not None:
            with_count += 1

total_count = no_count + with_count
print("Total:", total_count)
print("NO Space:", no_count)
print("With Space", with_count)

f.close()

Total: 2
NO Space: 1
With Space 1


In [24]:
'''
    [Binary Operator]
        Binary: '+', '-', '*', '/', '%', (Arithmetic)
                '=', '+=', '-=', '*=', '/=', '%=', 
                '&=', '^=', '|=', '<<=', '>>=', '>>>=',  (Assignment)
                '==', '!=', '>', '<', '>=', '<=',   (Comparison)
                '&&', '||', (Logical)
                '>>', '<<', '>>>' (Shift)

            주의해야 할 부분:   '-'는 부호를 의미하는 단항연산자와 구분이 필요
                                    -> 부호의 경우 앞에 문자가 오지 않을 듯 (= -1 + 2 와 같은 형태,,,)
                                        => \w(알파벳이나 숫자)로 구분해 줄 수 있을 듯
                                    -> 포인터 연산자 '->'에서도 구분이 필요
                                        => 이것도 \w로 거를 수 있을 듯

                                '*'는 포인터 연산자와 구분이 필요
                                    -> 아래의 경우는 구분할 필요 없을 듯
                                        int* a; vs int * a vs int *a
                                    -> 다만, *a와 같이 참조 연산은 구분할 필요 있음
                                        => 이것도 \w로 거를 수 있을 듯

                                '/'는 주석기호 '//'와의 구분이 필요
                                        => \w로 거를 수 있을 듯
                                
                                '='의 구분 필요
                                        => \w로 거를 수 있을 듯
            

'''

f = open('./optest.c', 'r')
line_list = f.readlines()

operator_list = [   '+', '-', '*', '/', '%',    # Arithmetic
                    '=', '+=', '-=', '*=', '/=', '%=', 
                    '&=', '^=', '|=', '<<=', '>>=', '>>>=', # Assignment
                    ',', '==', '!=', '>', '<', '>=', '<=',       # Comparison
                    '&&', '||',    # Logical
                    '>>', '<<', '>>>' ]     # Shift
          
no_operator_no_count = 0
no_opeartor_yes_count = 0
yes_operator_no_count = 0
yes_operator_yes_count = 0

for line in line_list:
    for operator in operator_list:
        no_operator_no = re.compile(rf'\w{operator}\w')
        no_operator_yes = re.compile(rf'\w{operator}\s+\w')
        #yes_operator_no = re.compile(rf'\w\s+{operator}\w')
        #yes_operator_yes = re.compile(rf'\w\s+{operator}\s+\w')
        
        if no_operator_no.search(line) is not None:
            no_operator_no_count += 1
            print(line)
        elif no_operator_yes.search(line) is not None:
            no_opeartor_yes_count += 1
        #elif yes_operator_no.search(line) is not None:
        #    yes_operator_no_count += 1
        #elif yes_operator_yes.search(line) is not None:
        #    yes_operator_yes_count += 1

total_count = no_operator_no_count + no_opeartor_yes_count + yes_operator_no_count + yes_operator_yes_count

print("Total:", total_count)
print("NO + NO Space:", no_operator_no_count)
print("NO + YES Space:", no_opeartor_yes_count)
print("YES + NO Space:", yes_operator_no_count)
print("YES + YES Space:", yes_operator_yes_count)

f.close()

#include <stdio.h>

#include <stdio.h>

#include <stdio.h>

#include <stdio.h>



int main(void)

int main(void)

int main(void)

int main(void)

{

    int a = 0;

    int a = 0;

    int a = 0;

    int a = 0;



    return 0;

    return 0;

    return 0;

    return 0;

}
Total: 21
NO + NO Space: 20
NO + YES Space: 1
YES + NO Space: 0
YES + YES Space: 0


This is for test